# Homework
The goal of this homework is to get familiar with tools like MLflow for experiment tracking and model management.

# Setup


## Import libraries

In [17]:
import requests
import os

## Define paths

In [14]:
# Directory of the raw data files
_data_root = './data'

# Create the dataset directory
!mkdir -p {_data_root}

PATH_RAW_DATA_JAN_2022 = f'{_data_root}/green_tripdata_2022-01.parquet'
PATH_RAW_DATA_FEB_2022 = f'{_data_root}/green_tripdata_2022-02.parquet'
PATH_RAW_DATA_MAR_2022 = f'{_data_root}/green_tripdata_2022-03.parquet'

URL_RAW_DATA_JAN_2022 = 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-01.parquet'
URL_RAW_DATA_FEB_2022 = 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-02.parquet'
URL_RAW_DATA_MAR_2022 = 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-03.parquet'

TAXI_DATA_FOLDER = _data_root

## Download data

In [15]:
def download_file(url: str, save_path: str) -> None:
    """Download any type of file and save it locally.

    Args:
        url (str): URL of data file
        save_path (str): Local path to save the file
    """

    response = requests.get(url)
    with open(save_path, 'wb') as file:
        file.write(response.content)

try:
    download_file(URL_RAW_DATA_JAN_2022, PATH_RAW_DATA_JAN_2022)
    download_file(URL_RAW_DATA_FEB_2022, PATH_RAW_DATA_FEB_2022)
    download_file(URL_RAW_DATA_MAR_2022, PATH_RAW_DATA_MAR_2022)
    print(f"Data downloaded, check the folder {_data_root}")
except Exception as e:
    print(f"Error: {e}")


Data downloaded, check the folder ./data


# Q1. Install the package
To get started with MLflow you'll need to install the appropriate Python package.

For this we recommend creating a separate Python environment, for example, you can use conda environments, and then install the package there with pip or conda.

Once you installed the package, run the command `mlflow --version`  and check the output.

What's the version that you have?

In [1]:
# Checking version
!mlflow --version

mlflow, version 2.3.2


# Q2. Download and preprocess the data
We'll use the Green Taxi Trip Records dataset to predict the amount of tips for each trip.

Download the data for January, February and March 2022 in parquet format from [here](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Use the script preprocess_data.py located in the folder homework to preprocess the data.

The script will:

* load the data from the folder <TAXI_DATA_FOLDER> (the folder where you have downloaded the data),
* fit a DictVectorizer on the training set (January 2022 data),
* save the preprocessed datasets and the DictVectorizer to disk.


Your task is to download the datasets and then execute this command:

```bash
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```

In [16]:
! python3 preprocess_data.py --raw_data_path {TAXI_DATA_FOLDER} --dest_path ./output

Tip: go to 02-experiment-tracking/homework/ folder before executing the command and change the value of <TAXI_DATA_FOLDER> to the location where you saved the data.

So what's the size of the saved DictVectorizer file?

In [21]:
def get_file_size(file_path):
    size_in_bytes = os.path.getsize(file_path)
    return size_in_bytes / 1000

# Usage
file_path = './output/dv.pkl'
file_size_in_kb = get_file_size(file_path)
print(f"File size: {file_size_in_kb} KB")

File size: 153.66 KB
